In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import precision_score, recall_score
import pandas as pd
from sklearn.metrics import f1_score, recall_score, precision_score, mean_absolute_error 
from tensorflow.keras import layers, models, applications
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Multiply, concatenate, Input, BatchNormalization, ReLU, Add, UpSampling2D
from tensorflow.keras.models import Model
import keras.backend as K
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Imports
import os
from tensorflow import keras
from glob import glob
from tqdm import tqdm 
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras.models import load_model
from keras.losses import binary_crossentropy 
from tensorflow.keras.layers import ReLU, Add  
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential 
#import albumentations as A
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, precision_recall_curve
from scipy.spatial.distance import directed_hausdorff
from tensorflow.keras.applications import EfficientNetB0
import matplotlib.cm as cm 
import tenseal as ts
import random
import time
 
tf.compat.v1.enable_eager_execution()

# Set the seed for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

set_seed(seed=42)
print("done")

2024-09-29 17:50:27.597469: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-29 17:50:28.077801: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 17:50:28.077858: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 17:50:28.079561: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 17:50:28.392770: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-29 17:50:28.394975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

done


In [2]:
# Generate mock training, validation, and test data

# Image size and other constants
IMG_SIZE = 240
batch_size = 32
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3) 
num_clients=5
epochs=100
#num_rounds=1
#local_epochs=1

train_image_dir = "/home/freno/Desktop/THESIS/JAJA-TA/images"
train_mask_dir = "/home/freno/Desktop/THESIS/JAJA-TA/masks"


test_image_dir = "/home/freno/Desktop/THESIS/JAJA-TA/TEST_ALL/general_test/images"
test_mask_dir = "/home/freno/Desktop/THESIS/JAJA-TA/TEST_ALL/general_test/masks"


In [3]:
import os
import tensorflow as tf

class DataLoader:
    def __init__(self, image_paths, mask_paths, target_shape=(IMG_SIZE, IMG_SIZE), augment=False):
        self.image_paths = sorted(image_paths)
        self.mask_paths = sorted(mask_paths)
        self.dataset = tf.data.Dataset.from_tensor_slices((self.image_paths, self.mask_paths))
        self.AUTOTUNE = tf.data.experimental.AUTOTUNE
        self.target_shape = target_shape
        self.augment = augment  # Set augment flag

    @tf.function
    def parse_images(self, image_path, mask_path):
        # Read and preprocess the image
        image = tf.io.read_file(image_path)
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.cast(image, tf.float32)
        image = tf.image.resize(image, self.target_shape) / 255.

        # Read and preprocess the mask
        mask = tf.io.read_file(mask_path)
        mask = tf.io.decode_jpeg(mask, channels=1)
        mask = tf.cast(mask, tf.float32)
        mask = tf.image.resize(mask, self.target_shape) / 255.

        # Apply data augmentation if augment flag is True
        if self.augment:
            image, mask = self.augment_image_mask(image, mask)

        return image, mask

    @tf.function 
    def data_generator(self, batch_size=batch_size, shuffle=True, repeat=False):
        dataset = self.dataset
        if shuffle:
            dataset = dataset.shuffle(buffer_size=len(self.image_paths))
    
        dataset = dataset.map(self.parse_images, num_parallel_calls=self.AUTOTUNE)
    
        if repeat:
            dataset = dataset.repeat()  # Use repeat only for training
    
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(buffer_size=self.AUTOTUNE)
        return dataset


    @tf.function
    def augment_image_mask(self, image, mask):
        """Apply data augmentation to image and mask."""
        # Random flip (horizontal and/or vertical)
        if tf.random.uniform(()) > 0.5:
            image = tf.image.flip_left_right(image)
            mask = tf.image.flip_left_right(mask)

        if tf.random.uniform(()) > 0.5:
            image = tf.image.flip_up_down(image)
            mask = tf.image.flip_up_down(mask)

        # Random rotation (90 degrees increments)
        k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
        image = tf.image.rot90(image, k)
        mask = tf.image.rot90(mask, k)

        return image, mask

def prepare_federated_data(image_dir, mask_dir, num_clients=5, target_shape=(IMG_SIZE, IMG_SIZE), augment=True):
    # Get image and mask paths
    image_paths = sorted([os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith(('.jpg', '.jpeg', '.png'))])
    mask_paths = sorted([os.path.join(mask_dir, fname) for fname in os.listdir(mask_dir) if fname.endswith(('.jpg', '.jpeg', '.png'))])
    
    # Initialize the DataLoader with all paths
    total_loader = DataLoader(image_paths, mask_paths, target_shape=target_shape, augment=augment)
    
    # Get the total dataset
    total_dataset = total_loader.dataset
    total_size = len(image_paths)
    
    # Shuffle the dataset before splitting
    total_dataset = total_dataset.shuffle(buffer_size=total_size)
    
    # Split the dataset into num_clients subsets
    client_datasets = []
    split_size = total_size // num_clients
    for i in range(num_clients):
        start_idx = i * split_size
        end_idx = (i + 1) * split_size if i != num_clients - 1 else total_size
        
        client_image_paths = image_paths[start_idx:end_idx]
        client_mask_paths = mask_paths[start_idx:end_idx]
        
        client_loader = DataLoader(client_image_paths, client_mask_paths, target_shape=target_shape, augment=augment)
        client_datasets.append(client_loader)

    return client_datasets

# Prepare the federated datasets for training
train_client_datasets = prepare_federated_data(train_image_dir, train_mask_dir, num_clients=num_clients, augment=True)
test_client_datasets = prepare_federated_data(test_image_dir, test_mask_dir, num_clients=num_clients, augment=False)



In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization
from tensorflow.keras.models import Model

def conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    """Basic convolutional block with two Conv2D layers and batch normalization."""
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = BatchNormalization()(c)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    c = BatchNormalization()(c)
    return c

def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    """Downsampling block: Conv2D + Conv2D + MaxPooling2D."""
    c = conv_block(x, filters, kernel_size, padding, strides)
    p = MaxPooling2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    """Upsampling block: Upsample + Conv2D + Concatenate with skip connections."""
    us = UpSampling2D((2, 2))(x)
    concat = Concatenate()([us, skip])
    c = conv_block(concat, filters, kernel_size, padding, strides)
    return c

def UNet():
    f = [4, 8, 16, 32, 64]  # Filter sizes for each level
    inputs = Input((240, 240, 3))

    # Downsampling path (Encoder)
    c1, t1 = down_block(inputs, f[0])
    c2, t2 = down_block(t1, f[1])
    c3, t3 = down_block(t2, f[2])
    c4, t4 = down_block(t3, f[3])

    # Bottleneck
    bn = conv_block(t4, f[4])

    # Nested connections (U-Net++)
    # Dense skip connections between the decoder and encoder
    u1_0 = up_block(bn, c4, f[3])
    u2_0 = up_block(u1_0, c3, f[2])
    u3_0 = up_block(u2_0, c2, f[1])
    u4_0 = up_block(u3_0, c1, f[0])

    # Extra skip connections for U-Net++
    u2_1 = up_block(u1_0, u2_0, f[2])
    u3_1 = up_block(u2_1, u3_0, f[1])
    u4_1 = up_block(u3_1, u4_0, f[0])

    u3_2 = up_block(u2_1, u3_1, f[1])
    u4_2 = up_block(u3_2, u4_1, f[0])

    u4_3 = up_block(u3_2, u4_2, f[0])

    # Final output layer
    outputs = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4_3)

    # Create model
    model = Model(inputs, outputs) 
    
    return model

# Instantiate the U-Net++ model
global_model = UNetPlusPlus()

In [5]:
# Function to Train on Each Client
def train_on_client(client_data, global_model, epochs=5, batch_size=32):
    # Clone the global model for client-specific training
    client_model = tf.keras.models.clone_model(global_model)
    client_model.set_weights(global_model.get_weights())  # Initialize with global model weights

    client_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                         loss="binary_crossentropy", metrics=['accuracy'])

    client_generator = client_data.data_generator(batch_size=batch_size)
    
    # Fit the client model and get the training loss
    history = client_model.fit(client_generator, epochs=epochs, steps_per_epoch=len(client_data.image_paths) // batch_size)

    # Return weight updates and training loss
    weight_updates = [client_model.get_weights()[i] - global_model.get_weights()[i] 
                      for i in range(len(global_model.get_weights()))]
    return weight_updates, history.history['loss'][-1]  # Return final loss of the last epoch
    

# Federated Averaging to Aggregate the Weights
def aggregate_models(global_model, client_weight_updates, num_clients):
    # Average the weight updates from all clients
    averaged_weight_updates = [np.mean([client_weight_updates[i][j] for i in range(num_clients)], axis=0) 
                               for j in range(len(global_model.get_weights()))]

    # Apply the averaged weight updates to the global model
    new_global_weights = [global_model.get_weights()[i] + averaged_weight_updates[i] 
                          for i in range(len(global_model.get_weights()))]
    global_model.set_weights(new_global_weights)
    

# Federated Learning Process with Loss Tracking
def federated_training(train_client_datasets, global_model, epochs=10, num_clients=5, batch_size=32):
    """
    Federated training with client weight updates and loss tracking.

    Args:
        train_client_datasets: List of DataLoader instances for each client.
        global_model: The global model to be trained.
        epochs: Number of global training epochs.
        num_clients: Number of clients.
        batch_size: Batch size for training.

    Returns:
        Updated global model and list of global training losses.
    """
    global_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                         loss="binary_crossentropy", metrics=['accuracy'])

    global_losses = []  # Track the global loss after each epoch

    for epoch in range(epochs):
        print(f"Global Training Epoch {epoch + 1}/{epochs}")
        client_weight_updates = []
        epoch_loss = 0  # To accumulate loss from all clients

        for i in range(num_clients):
            print(f"Training on client {i + 1}/{num_clients}")
            # Train the client and collect weight updates (deltas) and loss
            weight_updates, client_loss = train_on_client(train_client_datasets[i], global_model, epochs=1, batch_size=batch_size)
            client_weight_updates.append(weight_updates)
            epoch_loss += client_loss  # Accumulate client loss

        # Aggregate the model using weight updates
        aggregate_models(global_model, client_weight_updates, num_clients)
        print("Updated global model.")

        # Average the loss across clients and store for plotting
        avg_epoch_loss = epoch_loss / num_clients
        global_losses.append(avg_epoch_loss)
        print(f"Average Loss for Epoch {epoch + 1}: {avg_epoch_loss}")

    return global_model, global_losses



In [ ]:
# Initialize new model for tandard FedAvg
global_fedavg =UNetPlusPlus()

# Run standard FedAvg
print("Running Standard FedAvg...") 
start_time = time.time()
# Train the global model with FedAvg 
fedavg_model, fedavg_loss =  federated_training(train_client_datasets, 
                                                global_model, epochs=epochs, 
                                                num_clients=num_clients, batch_size=batch_size)
end_time = time.time()

# Store training time for Standard FedAvg
fl_training_times = end_time - start_time

print(f"Standard FedAvg training time: {fl_training_times:.2f} seconds")

 

Running Standard FedAvg...
Global Training Epoch 1/20
Training on client 1/5
394/394 [==============================] - 64s 129ms/step - loss: 0.4884 - accuracy: 0.8347
Training on client 2/5
394/394 [==============================] - 52s 107ms/step - loss: 0.5062 - accuracy: 0.7963
Training on client 3/5
394/394 [==============================] - 53s 109ms/step - loss: 0.5146 - accuracy: 0.8249
Training on client 4/5
394/394 [==============================] - 47s 95ms/step - loss: 0.5066 - accuracy: 0.7834
Training on client 5/5
396/396 [==============================] - 51s 105ms/step - loss: 0.5230 - accuracy: 0.7601
Updated global model.
Average Loss for Epoch 1: 0.5077752649784089
Global Training Epoch 2/20
Training on client 1/5
394/394 [==============================] - 52s 107ms/step - loss: 0.2724 - accuracy: 0.9081
Training on client 2/5
394/394 [==============================] - 47s 96ms/step - loss: 0.3011 - accuracy: 0.8928
Training on client 3/5
394/394 [=================

In [ ]:


# BB84 Quantum Key Distribution Protocol
class BB84:
    def __init__(self, key_length=128):
        self.key_length = key_length

    def generate_key(self):
        bits = [random.choice([0, 1]) for _ in range(self.key_length)]
        bases = [random.choice([0, 1]) for _ in range(self.key_length)]
        return bits, bases

    def receive_key(self, alice_bits, alice_bases, client_bases):
        key = [alice_bits[i] for i in range(self.key_length) if alice_bases[i] == client_bases[i]]
        return key

# Fully Homomorphic Encryption 
class FullyHomomorphicEncryption:
    def __init__(self, poly_modulus_degree=16384*2):  # Increased degree
        self.context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=poly_modulus_degree, coeff_mod_bit_sizes=[60, 40, 40, 60])
        self.context.generate_galois_keys()
        self.context.global_scale = 2**40

    def set_key(self, key):
        # Optionally use the key to further personalize encryption
        self.context.global_scale = 2**40  # Example of using a constant global scale

    def encrypt(self, tensor):
        if isinstance(tensor, np.ndarray):
            flat_tensor = tensor.flatten()
            encrypted_tensor = ts.ckks_vector(self.context, flat_tensor)
            return encrypted_tensor
        else:
            raise ValueError("Input should be a numpy array")

    def decrypt(self, encrypted_tensor, shape):
        flat_tensor = encrypted_tensor.decrypt()
        return np.reshape(flat_tensor, shape)

    def add_encrypted(self, enc1, enc2):
        return enc1 + enc2

    def multiply_encrypted(self, enc1, scalar):
        return enc1 * scalar

def train_on_client(client_data, global_model, epochs=5, batch_size=32):
    # Clone the global model for client-specific training
    client_model = tf.keras.models.clone_model(global_model)
    client_model.set_weights(global_model.get_weights())  # Initialize with global model weights

    client_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                         loss="binary_crossentropy", metrics=['accuracy'])

    client_generator = client_data.data_generator(batch_size=batch_size)
    
    # Fit the client model and get the training loss
    history = client_model.fit(client_generator, epochs=epochs, steps_per_epoch=len(client_data.image_paths) // batch_size)

    # Return weight updates and training loss
    weight_updates = [client_model.get_weights()[i] - global_model.get_weights()[i] 
                      for i in range(len(global_model.get_weights()))]
    return weight_updates, history.history['loss'][-1]  # Return final loss of the last epoch

# Federated Learning Process with FHE and BB84
def qce_training(train_client_datasets, global_model, epochs=10, num_clients=5, batch_size=32, key_length=128):
    """
    Federated training with BB84 quantum key distribution and Fully Homomorphic Encryption.

    Args:
        train_client_datasets: List of DataLoader instances for each client.
        global_model: The global model to be trained.
        epochs: Number of global training epochs.
        num_clients: Number of clients.
        batch_size: Batch size for training.
        key_length: Length of the quantum key (default is 128 bits).

    Returns:
        Updated global model and list of global training losses.
    """
    global_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                         loss="binary_crossentropy", metrics=['accuracy'])

    global_losses = []  # Track the global loss after each epoch
    bb84_protocol = BB84(key_length)  # Initialize BB84 protocol for quantum key distribution

    for epoch in range(epochs):
        print(f"Global Training Epoch {epoch + 1}/{epochs}")
        client_weight_updates = []
        epoch_loss = 0  # To accumulate loss from all clients

        for i in range(num_clients):
            print(f"Training on client {i + 1}/{num_clients}")
            # Step 1: Alice (server) generates bits and bases for BB84
            alice_bits, alice_bases = bb84_protocol.generate_key()

            # Step 2: Client generates its bases and receives the key
            client_bases = [random.choice([0, 1]) for _ in range(key_length)]
            client_key = bb84_protocol.receive_key(alice_bits, alice_bases, client_bases)

            # Step 3: Initialize Fully Homomorphic Encryption with the client's key
            fhe = FullyHomomorphicEncryption()
            fhe.set_key(client_key)

            # Step 4: Train the client and collect weight updates (deltas) and loss
            weight_updates, client_loss = train_on_client(train_client_datasets[i], global_model, epochs=1, batch_size=batch_size)
            epoch_loss += client_loss  # Accumulate client loss

            # Step 5: Encrypt the weight updates using FHE
            #encrypted_weight_updates = [fhe.encrypt(np.array(w)) for w in weight_updates]
            encrypted_weight_updates = [fhe.encrypt_in_batches(np.array(w), batch_size=batch_size) for w in weight_updates]
            client_weight_updates.append(encrypted_weight_updates)

        # Step 6: Aggregate the encrypted weight updates using FHE without decrypting them
        encrypted_aggregated_updates = []
        for j in range(len(global_model.get_weights())):
            enc_sum = client_weight_updates[0][j]  # Start with the first client's update
            for k in range(1, num_clients):
                enc_sum = fhe.add_encrypted(enc_sum, client_weight_updates[k][j])  # Add updates in encrypted space
            encrypted_aggregated_updates.append(fhe.multiply_encrypted(enc_sum, 1 / num_clients))  # Average

        # Step 7: Decrypt aggregated weight updates and apply them to the global model
        decrypted_updates = [fhe.decrypt(enc_update, global_model.get_weights()[j].shape) for j, enc_update in enumerate(encrypted_aggregated_updates)]
        new_global_weights = [global_model.get_weights()[i] + decrypted_updates[i] for i in range(len(global_model.get_weights()))]
        global_model.set_weights(new_global_weights)

        print("Updated global model.")

        # Average the loss across clients and store for plotting
        avg_epoch_loss = epoch_loss / num_clients
        global_losses.append(avg_epoch_loss)
        print(f"Average Loss for Epoch {epoch + 1}: {avg_epoch_loss}")

    return global_model, global_losses

 

In [ ]:
# Initialize new model for QKD and FHE
global_qcefl = UNetPlusPlus()
    
# Run FedAvg with QKD and FHE
print("QKD-FHE...")
    
start_time = time.time()
# Train the global model with FedAvg
qce_fl_model, qce_loss = qce_training(train_client_datasets, 
                                                global_model, epochs=epochs, 
                                                num_clients=num_clients, batch_size=batch_size, key_length=128)
end_time = time.time()

# Store training time for FedAvg with QKD and FHE
qce_training_times = end_time - start_time

print(f"QCE-FL training time: {qce_training_times:.2f} seconds")

 

In [ ]:
import matplotlib.pyplot as plt


epochs = range(1, len(qce_loss) + 1)

# Create the plot
plt.figure(figsize=(10, 6))

plt.plot(epochs, fedavg_loss, label='FedAvg Loss', marker='o', color='blue')
plt.plot(epochs, qce_loss, label='QCE-FL Loss', marker='o', color='red')

plt.xlabel('Epochs')
plt.ylabel('Training Loss')
#plt.title('Training Loss Comparison: QCE-FL vs FedAvg')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
 

def plot_computational_burden(fl_training_times, qce_training_times):
    # Define labels and values
    labels = ['FedAvg', 'QCE-FL']
    values = [fl_training_times, qce_training_times]
    
    # Define colors
    colors = ['#D3D3D3', '#FFFFE0']  # Light ash and light yellow

    # Create the bar chart
    plt.figure(figsize=(8, 6))
    plt.bar(labels, values, color=colors)
    
    # Add title and labels
    plt.title('Computational Burden: FedAvg vs QCE-FL')
    plt.ylabel('Training Time (seconds)')
    
    # Show the plot
    plt.show()

# Example usage
#fl_training_times = 50  # Example training time for FedAvg
#qce_training_times = 120  # Example training time for QCE-FL
plot_computational_burden(fl_training_times, qce_training_times)


In [ ]:
# Function to create a combined test dataset
def create_combined_test_dataset(test_client_datasets):
    combined_image_paths = []
    combined_mask_paths = []

    # Iterate over each client's test dataset to combine the paths
    for client_data in test_client_datasets:
        combined_image_paths.extend(client_data.image_paths)
        combined_mask_paths.extend(client_data.mask_paths)

    # Create a new DataLoader for the combined test dataset
    combined_dataset = DataLoader(combined_image_paths, combined_mask_paths, augment=False)
    return combined_dataset

 
# Create the combined test dataset
combined_test_dataset = create_combined_test_dataset(test_client_datasets)

In [ ]:
# Function to extract ground truth masks from the combined test dataset
def extract_ground_truth_masks(data_loader):
    masks = []

    for image_batch, mask_batch in data_loader.data_generator(batch_size=1, shuffle=False):
        masks.append(mask_batch.numpy())  # Convert tensor to numpy array

    # Stack the masks to form a single array
    return np.vstack(masks)  # Use vstack to concatenate along the first axis

# Get ground truth masks
ground_truth_masks = extract_ground_truth_masks(combined_test_dataset)

In [ ]:
 

def dice_score(predictions, masks, threshold=0.5):
    """
    Calculate the DICE score for the given predictions and ground truth masks.
    
    Args:
        predictions: Model predictions (numpy array).
        masks: Ground truth masks (numpy array).
        threshold: Threshold to binarize predictions.
    
    Returns:
        DICE score.
    """
    predictions_binary = (predictions > threshold).astype(np.float32)
    intersection = np.sum(predictions_binary * masks)
    return 2 * intersection / (np.sum(predictions_binary) + np.sum(masks) + 1e-8)  # Add epsilon to avoid division by zero

# Predicting on the test datasets
def predict_on_test(test_data, global_model, batch_size=32):
    test_generator = test_data.data_generator(batch_size=batch_size, shuffle=False)
    predictions = global_model.predict(test_generator)
    return predictions


 
# Get predictions for both models
predictions_fedavg = predict_on_test(combined_test_dataset, fedavg_model)
predictions_qce_fl = predict_on_test(combined_test_dataset, qce_fl_model)

# Calculate DICE scores
dice_score_fedavg = dice_score(predictions_fedavg, ground_truth_masks)
dice_score_qce_fl = dice_score(predictions_qce_fl, ground_truth_masks)

# Plot DICE scores
models = ['FedAvg', 'QCE-FL']
dice_scores = [dice_score_fedavg, dice_score_qce_fl]

plt.figure(figsize=(8, 5))
plt.bar(models, dice_scores, color=['lightyellow', 'lightcoral'])
plt.ylabel('DICE Score')
#plt.title('DICE Scores of the Models')
plt.ylim(0, 1)  # DICE scores range from 0 to 1
plt.show()
 

In [ ]:
fl_models = {
    'FedAvg': fedavg_model,
    'QCE-FL': qce_fl_model
}

# Assuming combined_test_dataset is your DataLoader instance
# Make sure you get the original images in the correct format
original_images = [test_data[0] for test_data in combined_test_dataset.data_generator(batch_size=len(combined_test_dataset.image_paths), shuffle=False)]
 

In [ ]:
 

def visualize_results(images, ground_truth, models, num_samples=2):
    """
    Visualizes the segmentation results for the specified number of samples.

    Args:
        images (list): List of original images.
        ground_truth (np.ndarray): Array of ground truth masks.
        models (dict): Dictionary of models to evaluate.
        num_samples (int): Number of samples to visualize.
    """
    num_models = len(models)

    # Determine how many samples to visualize based on available data
    num_samples = min(num_samples, len(images))

    # Create a figure with adjusted spacing
    fig, axes = plt.subplots(
        num_samples, 
        num_models + 2,  # 2 for original and ground truth, plus models
        figsize=(15, 5 * num_samples), 
        gridspec_kw={'wspace': 0.01, 'hspace': 0.1}
    )

    for i in range(num_samples):
        # Convert image to numpy array (assuming images are still Tensors)
        image_np = images[i].numpy() if isinstance(images[i], tf.Tensor) else images[i]
        ground_truth_np = ground_truth[i]  # No need to call .numpy() here

        # Display original image
        ax = axes[i, 0] if num_samples > 1 else axes[0]
        ax.imshow(image_np[0], cmap='gray')  # Display the first image in the batch
        ax.axis('off')
        if i == num_samples - 1:
            ax.set_title("Image", y=-0.2, fontsize=14)
        
        # Display ground truth mask overlaid on the original image
        ax = axes[i, 1] if num_samples > 1 else axes[1]
        ax.imshow(image_np[0], cmap='gray')  # Show original image
        ax.imshow(ground_truth_np.squeeze(), cmap='jet', alpha=0.5)  # Overlay mask
        ax.axis('off')
        if i == num_samples - 1:
            ax.set_title("GT", y=-0.2, fontsize=14)
        
        # Display predicted masks overlaid on the original image for each model
        for j, (model_name, model) in enumerate(models.items()):
            # Predict and ensure it is a numpy array
            predicted_mask = model.predict(tf.expand_dims(image_np[0], axis=0), verbose=0)[0]
            if isinstance(predicted_mask, tf.Tensor):
                predicted_mask = predicted_mask.numpy()
            
            ax = axes[i, j + 2] if num_samples > 1 else axes[j + 2]
            ax.imshow(image_np[0], cmap='gray')  # Show original image
            ax.imshow(predicted_mask.squeeze(), cmap='jet', alpha=0.5)  # Overlay predicted mask
            ax.axis('off')
            if i == num_samples - 1:
                ax.set_title(model_name, y=-0.2, fontsize=14)
            
    # Adjust subplot spacing
    plt.subplots_adjust(wspace=0.1, top=0.9, hspace=0.1)
    plt.show()

 
# Call the function to visualize results
visualize_results(original_images, ground_truth_masks, fl_models, num_samples=4)
